In [ ]:

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os, json

OUTDIR = '/mnt/data/karate_modularity_outputs'
os.makedirs(OUTDIR, exist_ok=True)

plt.rcParams['figure.dpi'] = 120


In [ ]:

G = nx.karate_club_graph()  # nodes are 0..33
n = G.number_of_nodes()
m = G.number_of_edges()
print(f'Loaded Karate Club graph: n={n}, m={m}')

pos = nx.spring_layout(G, seed=42)  # fixed seed so layout doesn't change across iterations

_layout_file = os.path.join(OUTDIR, 'layout.json')
with open(_layout_file, 'w') as f:
    json.dump({str(k): list(map(float,v)) for k,v in pos.items()}, f)
print('Saved layout to', _layout_file)


In [ ]:

import math
def adjacency_matrix(G, nodes=None):
    if nodes is None:
        nodes = list(G.nodes())
    idx = {node:i for i,node in enumerate(nodes)}
    A = np.zeros((len(nodes), len(nodes)), dtype=float)
    for u, v in G.edges():
        if u in idx and v in idx:
            A[idx[u], idx[v]] = 1
            A[idx[v], idx[u]] = 1
    return A, nodes

def modularity_matrix(G, nodes=None):
    A, nodes_order = adjacency_matrix(G, nodes)
    full_deg = np.array([G.degree(n) for n in nodes_order], dtype=float)
    two_m = G.number_of_edges() * 2.0
    kkT = np.outer(full_deg, full_deg) / two_m
    B = A - kkT
    return B, nodes_order, A, full_deg

def leading_eigvec(B):
    vals, vecs = np.linalg.eigh(B)
    idx_max = np.argmax(vals)
    return float(vals[idx_max]), vecs[:, idx_max]


In [ ]:

from collections import deque, defaultdict

def spectral_bisection_recursive(G):
    communities = [list(G.nodes())]
    snapshots = []
    snapshots.append([list(c) for c in communities])

    queue = deque([communities[0]])
    iteration = 0
    while queue:
        C = queue.popleft()
        B, nodes_order, A, full_deg = modularity_matrix(G, C)
        lam, vec = leading_eigvec(B)
        if lam <= 1e-12:
            continue
        s = np.array([1 if val > 0 else -1 for val in vec])
        Cplus = [nodes_order[i] for i in range(len(nodes_order)) if s[i] == 1]
        Cminus = [nodes_order[i] for i in range(len(nodes_order)) if s[i] == -1]
        if len(Cplus) == 0 or len(Cminus) == 0:
            continue
        for i,comm in enumerate(communities):
            if set(comm) == set(C):
                communities[i] = Cplus
                communities.insert(i+1, Cminus)
                break
        queue.append(Cplus)
        queue.append(Cminus)
        iteration += 1
        snapshots.append([list(c) for c in communities])
    return snapshots

snapshots = spectral_bisection_recursive(G)
print(f'Found {len(snapshots[-1])} communities after recursion. Iterations (snapshots): {len(snapshots)}')


In [ ]:

from itertools import cycle
color_cycle = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

metric_names = ['degree', 'betweenness', 'closeness', 'clustering']
metrics_over_iterations = {name: [] for name in metric_names}
partitions = snapshots  # each element is a list of communities

def compute_metrics_for_partition(G, partition):
    node_metrics = {name: {} for name in metric_names}
    for comm in partition:
        sub = G.subgraph(comm).copy()
        deg = nx.degree_centrality(sub)
        btw = nx.betweenness_centrality(sub, normalized=True)
        clo = nx.closeness_centrality(sub)
        clust = nx.clustering(sub)
        for node in sub.nodes():
            node_metrics['degree'][node] = deg.get(node, 0.0)
            node_metrics['betweenness'][node] = btw.get(node, 0.0)
            node_metrics['closeness'][node] = clo.get(node, 0.0)
            node_metrics['clustering'][node] = clust.get(node, 0.0)
    return node_metrics

for it, partition in enumerate(partitions):
    fig, ax = plt.subplots(figsize=(6,5))
    node_color = {}
    for c_idx, comm in enumerate(partition):
        col = color_cycle[c_idx % len(color_cycle)]
        for node in comm:
            node_color[node] = col
    colors = [node_color[n] for n in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=200, ax=ax)
    nx.draw_networkx_labels(G, pos, labels={n:str(n) for n in G.nodes()}, font_size=8, ax=ax)
    nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax)
    ax.set_title(f'Iteration {it}: {len(partition)} communities')
    ax.axis('off')
    fname = os.path.join(OUTDIR, f'partition_iter_{it}.png')
    fig.savefig(fname, bbox_inches='tight')
    plt.close(fig)

    node_metrics = compute_metrics_for_partition(G, partition)
    for name in metric_names:
        metrics_over_iterations[name].append(node_metrics[name])

with open(os.path.join(OUTDIR, 'metrics_over_iterations.json'), 'w') as f:
    json.dump({k: [{str(node): float(v) for node,v in d.items()} for d in lst] for k,lst in metrics_over_iterations.items() for k in [k]}, f, indent=2)
print('Saved partition images and metrics. Number of iterations:', len(partitions))


In [ ]:

def plot_metric_evolution(metric_name, metrics_over_iterations, outdir=OUTDIR):
    data = metrics_over_iterations[metric_name]
    iterations = list(range(len(data)))
    node_series = {}
    for it, d in enumerate(data):
        for node in G.nodes():
            node_series.setdefault(node, []).append(d.get(node, 0.0))
    fig, ax = plt.subplots(figsize=(8,5))
    for node, series in node_series.items():
        ax.plot(iterations, series, label=str(node), linewidth=1, alpha=0.8)
    ax.set_xlabel('Iteration')
    ax.set_ylabel(metric_name)
    ax.set_title(f'Evolution of {metric_name} per node across iterations')
    top_nodes = sorted(node_series.keys(), key=lambda n: max(node_series[n]), reverse=True)[:6]
    for n in top_nodes:
        ax.plot(iterations, node_series[n], linewidth=2.2, label=f'Node {n} (top)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small')
    fname = os.path.join(outdir, f'metric_evolution_{metric_name}.png')
    fig.savefig(fname, bbox_inches='tight')
    plt.close(fig)
    return fname

plot_files = {}
for name in metric_names:
    plot_files[name] = plot_metric_evolution(name, metrics_over_iterations)
print('Saved metric evolution plots:', plot_files)


In [ ]:

def top_k_nodes_each_iteration(metric_dicts, k=3):
    top_counts = {}
    for d in metric_dicts:
        sorted_nodes = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
        topk = [n for n,_ in sorted_nodes[:k]]
        for n in topk:
            top_counts[n] = top_counts.get(n, 0) + 1
    return top_counts

discussion = {}
for name in metric_names:
    counts = top_k_nodes_each_iteration(metrics_over_iterations[name], k=3)
    sorted_counts = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)
    discussion[name] = sorted_counts

summary_lines = []
summary_lines.append('## Automated summary of central nodes across iterations\n')
for name, ranked in discussion.items():
    summary_lines.append(f'**Metric: {name}**')
    if not ranked:
        summary_lines.append('No data.')
        continue
    best = ranked[:5]
    summary_lines.append('Top nodes by frequency in top-3 across iterations:')
    for node, cnt in best:
        summary_lines.append(f'- Node {node}: appeared in top-3 in {cnt} iterations')
    summary_lines.append('')

summary_text = '\n'.join(summary_lines)
print(summary_text)

with open(os.path.join(OUTDIR, 'discussion_summary.txt'), 'w') as f:
    f.write(summary_text)
with open(os.path.join(OUTDIR, 'discussion_summary.json'), 'w') as f:
    json.dump(discussion, f, indent=2)
print('Saved discussion summary to outputs.')


In [ ]:

print('\nWhen you run this notebook top-to-bottom in a Python environment with networkx, numpy and matplotlib,')
print('it will produce:')
print('- partition images partition_iter_*.png for each iteration,')
print('- metric evolution plots metric_evolution_*.png,')
print('- metrics_over_iterations.json and discussion_summary.txt in the outputs folder.')
print('All saved under /mnt/data/karate_modularity_outputs/.')
